<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/neural_networks/RBN/rbn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Radial Basis Network

In [2]:
import numpy as np
import pandas as pd
from keras.layers import Layer, Dense, Flatten
from keras import backend as K
from keras.models import Sequential
from keras.losses import binary_crossentropy

In [11]:
X = np.load('./k49-train-imgs.npz')['arr_0']
y = np.load('./k49-train-labels.npz')['arr_0']
y = (y <= 25).astype(int)

print(X.shape)
print(y.shape)

(232365, 28, 28)
(232365,)


In [ ]:
class RBFLayer(Layer):
  def __init__(self, units, gamma, **kwargs):
    super(RBFLayer, self).__init__(**kwargs)
    self.units = units
    self.gamma = K.cast_to_floatx(gamma)

  def build(self, input_shape):
    self.mu = self.add_weight(name='mu',
                              shape=(int(input_shape[1]), self.units),
                              initializer='uniform',
                              trainable=True)
    super(RBFLayer, self).build(input_shape)
  
  def call(self, inputs):
    diff = K.expand_dims(inputs) - self.mu
    l2 = K.sum(K.pow(diff, 2), axis=1)
    res = K.exp(-1 * self.gamma * l2)
    return res

In [3]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(RBFLayer(units=10, gamma=0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss=binary_crossentropy)

In [12]:
model.fit(X, y, batch_size=256, epochs=3)

Epoch 1/3
908/908 [==============================] - 20s 21ms/step - loss: 0.6823
Epoch 2/3
908/908 [==============================] - 19s 21ms/step - loss: 0.6806
Epoch 3/3
908/908 [==============================] - 19s 21ms/step - loss: 0.6806
